In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("idealproblemsolver.pdf")
data = loader.load()

In [5]:
data

[Document(metadata={'source': 'idealproblemsolver.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'idealproblemsolver.pdf', 'page': 1}, page_content='THE IDEAL PROBLEM SOLVER '),
 Document(metadata={'source': 'idealproblemsolver.pdf', 'page': 2}, page_content='THE IDEAL PROBLEM SOLVER \nA Guide for Improving Thinking, \nLearning, and Creativity \nSecond Edition \nJohn D. Bransford \nBarry S. Stein \nrn \nW. H. Freeman and Company \nNew York '),
 Document(metadata={'source': 'idealproblemsolver.pdf', 'page': 3}, page_content="Library of Congress Cataloging-in-Publication Data \nBransford, John . \nThe ideal problem solver : a guide for improving thinking, \nlearning, and creativity I John D. Bransford, Barry S. Stein.- 2nd \ned. \np. em. \nIncludes bibliographical references and indexes. \nISBN 0-7167-2204-6 (cloth).- ISBN 0-7167-2205-4 (pbk .) \n1 . Problem solving . 2 . Thought and thinking . 3 . Creative \nability. 4. Learning, Psychology of. I. Stein, Barry S . \nI

In [6]:
len(data)

269

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
print("Total number of documents: ",len(docs))

Total number of documents:  721


In [10]:
docs[122]

Document(metadata={'source': 'idealproblemsolver.pdf', 'page': 47}, page_content="six-six weighing does not tell them which side the oddball is on. Note that \nwithout actively trying out their initial strategy and looking at the effects, \nthe definition of oddball would not be adequately understood. (You may \nnow want to continue to try to solve the oddball problem. The answer is \nin Appendix A . ) \nIn addition to looking at the effects of one's actions or decisions, it is \nimportant to learn from the experience . Ideally, one learns something \ngeneral so that subsequent attempts at problem solving can be improved. \nFor example, imagine that you have created a new advertisement for a \nproduct and had attempted to anticipate its effects by using a focus group \nof businessmen and women . When the ad actually airs on television, you \ndiscover a very negative reaction to it by various groups of people, espe\xad\ncially those who are out of work and unable to find a job . As a re

In [12]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()



True

In [15]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello world")
vector[:5]

[0.04909781739115715,
 -0.044328317046165466,
 -0.025365281850099564,
 -0.030721040442585945,
 0.019068587571382523]

In [17]:
vectorstore = Chroma.from_documents(documents=docs,embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [19]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":10})
retrieved_docs = retriever.invoke("How to solve the problem related to sleep insomnia?")

In [22]:
len(retrieved_docs)

10

In [21]:
print(retrieved_docs[5].page_content)

IIIII The Ideal Problem Solver 
back at the first sentence of the passage, you will see that the 
answer is quite familiar to you . 
2. An acrostic for remembering the cranial nerves that has been 
around for some time is "On old Olympus' towering tops a 
Finn and German vend some hops." 
3. A helpful acrostic for remembering whether to set one's clock 
forward or back an hour is "Spring forward and fall back." 
4 . You could combine the rhyming peg-word system (one is a 
bun, two is a shoe, and so on) and an acrostic in the follow­
ing manner. Think of the acrostic "You must wear two shoes 
to get in the door and stand in line." From the peg-word sys­
tem you know that "two is a shoe" (so two shoes equals 22). 
Similarly, you know from the peg-word system that "four is a 
door" (so door helps you remember the number 4) and that 
"nine is a line" (so line helps you remember 9). 
5. You could use acrostics such as "Fir is smooth" and "A rough


In [23]:
retrieved_docs

[Document(metadata={'page': 33, 'source': 'idealproblemsolver.pdf'}, page_content='A M odel tor Improving Problem - Solv ing Skills .. \nMEDICINE CLOCK. Set this little pill-box alarm to ring DON\'T LOCK YOURSELF OUT \nevery 1 / 2, l, 2, 4, 8, or 12 hours, and it plays a little song OF HOUSE, CAR! Magnetic cases \nto remind you to take your medicine . hide spare keys safely. \nA QUICK TWIST opens the most \nstubborn jars and bottles! \nSOLAR-POWERED CAR VENTILATOR. \nReduces heat build-up without leaving \nwindows open. \nFigure 5 Inventions designed to solve some common problems. \n3 A.M. and a man asks, "Is this Home Pac Pizza?" What should you do? If \nyou simply say, "No, you have the wrong number," you will have failed \nto anticipate the problem the caller wi ll face as soon as he hangs up: \nknowing whether the number itself was wrong or he had mere ly mis\xad\ndialed. \nSeveral years ago, after moving and receiving a new phone number, \none of us began to receive ca ll s at a l

In [41]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.0,max_tokens=600)

In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [43]:
system_prompt = (
    """
You are a compassionate and empathetic psychology assistant trained to help users navigate day-to-day life challenges. Your goal is to understand their issues, provide helpful guidance, and suggest ways to manage their emotions and actions in a constructive way.

Example issues you may help with include:
- Stress at work or school
- Difficulty managing relationships with family or friends
- Struggles with self-esteem or self-care routines
- Motivation and goal-setting advice
- Coping mechanisms for negative emotions like anxiety, frustration, or sadness

Always begin your response with empathy and validation of the user's feelings. Then offer practical advice or insights tailored to their specific situation.

---
{context}


"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)


In [46]:
response = rag_chain.invoke({"input":"How to solve the problem of bullying at school? and I am this since 1 month"})
print(response["answer"])

It's understandable that you're feeling distressed about being bullied for a month.  No one deserves to be treated that way, and it takes courage to reach out for help.  I'm here to support you.

Bullying is a complex problem, and there's no single "magic bullet" solution. However, here are some strategies you can try, broken down into immediate actions, long-term strategies, and when to seek further help:

**Immediate Actions:**

* **Ensure your safety:** If you feel physically threatened, remove yourself from the situation immediately and find a safe place, like a teacher's office or the library.
* **Don't engage:**  Bullies often thrive on reactions.  Try your best to remain calm and avoid responding. This can be incredibly difficult, but it can sometimes disarm the bully.  If you can, walk away.
* **Document everything:** Keep a record of each incident, including dates, times, locations, what happened, and who was involved. This documentation can be helpful if you need to report th